*import Libraries*

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import plotly.express as px

## 2. **Data Loading and exploration and cleaning**
 ↪ Load the csv file with the pandas
 
 ↪ creating the dataframe and understanding the data present in the dataset using pandas
 
 ↪ Dealing with the missing data, outliers and the incorrect records

In [3]:
df = pd.read_csv('./datasets/National Universities Rankings.csv')


- Viewing the data

In [4]:
df

,index,Name,Location,Rank,Description,Tuition and fees,In-state,Undergrad Enrollment
0,0,Princeton University,"Princeton, NJ",1,"Princeton, the fourth-oldest college in the Un...","$45,320",NaN,"5,402"
1,1,Harvard University,"Cambridge, MA",2,"Harvard is located in Cambridge, Massachusetts...","$47,074",NaN,"6,699"
2,2,University of Chicago,"Chicago, IL",3,"The University of Chicago, situated in Chicago...","$52,491",NaN,"5,844"
3,3,Yale University,"New Haven, CT",3,"Yale University, located in New Haven, Connect...","$49,480",NaN,"5,532"
4,4,Columbia University,"New York, NY",5,"Columbia University, located in Manhattan's Mo...","$55,056",NaN,"6,102"
...,...,...,...,...,...,...,...,...
226,226,University of Massachusetts--Dartmouth,"North Dartmouth, MA",220,"Located about 60 miles south of Boston, the Un...","$19,270","$12,588","7,295"
227,227,University of Missouri--St. Louis,"St. Louis, MO",220,Undergraduates at University of Missouri--St. ...,"$26,277","$10,065","13,569"
228,228,University of North Carolina--Greensboro,"Greensboro, NC",220,University of North Carolina--Greensboro is lo...,"$21,595","$6,733","15,951"
229,229,University of Southern Mississippi,"Hattiesburg, MS",220,The University of Southern Mississippi has two...,"$16,094","$7,224","11,840"


In [5]:
# hide all warnings runtime
import warnings
warnings.filterwarnings('ignore')

- let's see the exact column names which can be easily copied later on from National Universities Rankings

In [6]:
df.columns

Index(['index', 'Name', 'Location', 'Rank', 'Description', 'Tuition and fees',
       'In-state', 'Undergrad Enrollment'],
      dtype='object')

- Let's have a look on the shape of the dataset

In [7]:
df.shape

(231, 8)

Not enough, let's have a look on the columns and their data types using detailed info function

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   index                 231 non-null    int64 
 1   Name                  231 non-null    object
 2   Location              231 non-null    object
 3   Rank                  231 non-null    int64 
 4   Description           231 non-null    object
 5   Tuition and fees      231 non-null    object
 6   In-state              133 non-null    object
 7   Undergrad Enrollment  231 non-null    object
dtypes: int64(2), object(6)
memory usage: 14.6+ KB


# **Observations**
---
1. There are 231 rows and 8 columns in the dataset
2. The columns are of different data types
3. The columns in the datasets are:
   - `'index', 'Name', 'Location', 'Rank', 'Description', 'Tuition and fees',
       'In-state', 'Undergrad Enrollment'`
4. There are some missing values in the dataset which we will read in details and deal later on in the notebook.


In [9]:
# This column won't provide any value
df = df.drop('Description', axis=1)

In [10]:
df.describe()

,index,Rank
count,231.000000,231.000000
mean,115.000000,113.982684
std,66.828138,65.995518
min,0.000000,1.000000
25%,57.500000,56.000000
50%,115.000000,111.000000
75%,172.500000,171.000000
max,230.000000,220.000000


## Observations:
---
- We have only 2 columns as numeric data type, rest all are object data type (according to python),but we can see ,according to our needs.
---
- Let's check the null values 

In [11]:
df.isnull().sum()

index                    0
Name                     0
Location                 0
Rank                     0
Tuition and fees         0
In-state                98
Undergrad Enrollment     0
dtype: int64

In [12]:
#check percentage of null values 
df.isnull().sum()*100/len(df)

index                    0.000000
Name                     0.000000
Location                 0.000000
Rank                     0.000000
Tuition and fees         0.000000
In-state                42.424242
Undergrad Enrollment     0.000000
dtype: float64

## Observations:
---
- we have many null values in dataset, So first of all we deal with these null values 
---
- Let's remove or deal with null values in that columns `'In-state'`.

- we have 98 null values in that column`'In-state'`, So let's deal with these  values 

In [13]:
# convert data type of 'Tuition and fees'
def to_numeric(x):
    return int(x.replace(',', ''))

df['Tuition and fees'] = df['Tuition and fees'].str[1:].apply(to_numeric)

def to_numeric(y):
    return int(y.replace(',', ''))
df['Undergrad Enrollment'] = df['Undergrad Enrollment'].str[1:].apply(to_numeric)

In [14]:
# The amount of discount won't be as valuable as the fact that there's a discount or not
df["In-state-discount"] = (~df["In-state"].isnull())
df = df.drop("In-state", axis=1)

In [15]:
# We will only keep the State
df["State"] = df.Location.str[-2:]
df = df.drop('Location', axis=1)

- transform data and groupby of these columns

In [16]:
df["Tuition and fees"] = df["Tuition and fees"].astype(float)
df["State-mean-tuition-fees"] = df.groupby('State')['Tuition and fees'].transform('mean')
df["State-mean-rankings"] = df.groupby('State')['Rank'].transform('mean')

In [17]:
df['quantile-rank'] = pd.qcut(df['Rank'], 4, labels=["great reputation", "good reputation", "average reputation",
"low reputation"])
df['quantile-tuition-fees'] = pd.qcut(df['Tuition and fees'], 4, labels=["low", "average",
 "expensive", "very expensive"])

----
- create a box plot of "Tuition and fees",and "state"

In [18]:
fig = px.box(df.sort_values(by=['State-mean-tuition-fees'], ascending=False), x="State", y="Tuition and fees")
fig.show()

- create a box plot of "State",and "Rank

In [19]:
fig = px.box(df.sort_values(by=['State-mean-rankings']), x="State", y="Rank")
fig.show()

- The tendency is clear, those states that have the highest ranked universities also tend to have the most expensive ones.
----
- using 'groupby' function create a heatmap of that columns 'quantile-rank', 'quantile-tuition-fees'

In [20]:
heatmap_df = df[['quantile-rank', 'quantile-tuition-fees']]
heatmap_df["count"] = heatmap_df.groupby(['quantile-rank', 'quantile-tuition-fees'])['quantile-rank'].transform('count')
heatmap_df = heatmap_df.drop_duplicates()
fig = px.density_heatmap(heatmap_df,
                x='quantile-rank',
                y='quantile-tuition-fees',
                z='count'
               )
fig.show()

- Just like it was observed in the previous boxplot graphs, universities with a high reputation also tend to be more expensive.
## Observations
*There are a few reasons why universities that are higher in rankings tend to have higher tuition fees. First, higher-ranked universities often have a reputation for offering a higher quality education, which can make them more attractive to students. As a result, these universities may be able to charge higher tuition fees because students are willing to pay more for a better education. In addition, higher-ranked universities may also offer more specialized programs or facilities, such as state-of-the-art laboratories or research facilities, which can also contribute to their higher costs. Overall, there are many factors that can contribute to higher tuition fees at higher-ranked universities.*

---
- craate a scatter plot of that column "Tuition and fees", and "Undergrad Enrollment"

In [21]:
fig = px.scatter(df, x="Tuition and fees", y="Undergrad Enrollment", color="State",
hover_name="Name", size_max=20, trendline="ols", trendline_scope="overall")
figsize="Tuition and fees"
fig.show()

- The pattern the chart above shows is that universities that enroll more students tend to have lower tuition fees.
## Observation
*It is generally the case that larger universities have lower tuition fees because they have more students and therefore more revenue. This allows them to offer lower prices to their students. Additionally, larger universities often have more resources, such as state funding or private donations, which can also help to keep costs down.*

---

In [22]:
# create heatmap
heatmap_df = df[['quantile-rank', 'In-state-discount']]
heatmap_df["count"] = heatmap_df.groupby(['quantile-rank', 'In-state-discount'])['quantile-rank'].transform('count')
heatmap_df = heatmap_df.drop_duplicates()
fig = px.density_heatmap(heatmap_df,
                x='quantile-rank',
                y='In-state-discount',
                z='count'
               )
fig.show()

- The heatmap displayed above shows how there is a negative correlation between the reputation of a university and the likeliness that it provides a discount for in-state students.
- ## Observation
- *There are a few reasons why some reputable universities in the United States may not give in-state students a discount on their tuition fees. One reason is that these universities may have a high demand for their programs and can therefore afford to charge the same tuition fee to all students, regardless of their state of residence. Additionally, some of these universities may receive a significant amount of funding from donors and endowments, which allows them to offer a high-quality education without needing to offer discounts to in-state students*
---

In [23]:
# create scatter plot 
fig = px.scatter(df, x="Rank", y="Tuition and fees",
            size="Undergrad Enrollment", color="State",
                 hover_name="Name", size_max=20, trendline="ols", trendline_scope="overall")
fig.show()

- ## observation
- This final scatter plot shows that tuition fees are higher for more reputable universities and also confirms how universities with less students enrolled tend to be more expensive.

In [24]:
#   sunburst chart
df_sunburst = df.groupby(["Rank", "Tuition and fees"]).size().reset_index(name='counts')

fig11 = px.sunburst(df_sunburst, path=["Rank", "Tuition and fees"], values='counts')
fig11.show()

# save this plot as html
# fig11.write_html("./output/fig11.html")

#save this plot
# fig11.write_image("./output/fig11.png", scale=3)

- ## observation
- *This sunburst chart shows more clearly that tuition fees are higher for more reputable universities and also confirms how universities with less students enrolled tend to be more expensive.*